# Distribution-abundance Predict

**Written by Timm Nawrocki**

*Last updated Sunday November 17, 2019.*

In [ ]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Distribution-abundance Predict
# Author: Timm Nawrocki, Alaska Center for Conservation Science
# Created on: 2018-10-27
# Usage: Must be executed as a Jupyter Notebook in an Anaconda 3 installation.
# Description: "Distribution-Abundance Predict" applies the trained classifier and regressor to data in regular point grid format stored in csv files to create a composite prediction representing the distribution and proportional abundance of the target species.
# ---------------------------------------------------------------------------

In [ ]:
# Define model folder
model_folder = '/home/twnawrocki/modelResults/carex_aquatilis/'
# Define watershed data folder
watershed_folder = '/home/twnawrocki/watershedData/'
# Define prediction folder
prediction_folder = '/home/twnawrocki/predictions/carex_aquatilis/'

In [ ]:
# Define variable sets
predictor_all = ['compoundTopographic', 'dateFreeze_2000s', 'dateThaw_2000s', 'elevation', 'floodplainsDist', 'growingSeason_2000s', 'heatLoad', 'integratedMoisture', 'precipAnnual_2000s', 'roughness', 'siteExposure', 'slope', 'streamLargeDist', 'streamSmallDist', 'summerWarmth_2000s', 'surfaceArea', 'surfaceRelief', 'aspect', 'may_1_ultraBlue', 'may_2_blue', 'may_3_green', 'may_4_red', 'may_5_nearInfrared', 'may_6_shortInfrared1', 'may_7_shortInfrared2', 'may_evi2', 'may_nbr', 'may_ndmi', 'may_ndsi', 'may_ndvi', 'may_ndwi', 'june_1_ultraBlue', 'june_2_blue', 'june_3_green', 'june_4_red', 'june_5_nearInfrared', 'june_6_shortInfrared1', 'june_7_shortInfrared2', 'june_evi2', 'june_nbr', 'june_ndmi', 'june_ndsi', 'june_ndvi', 'june_ndwi', 'july_1_ultraBlue', 'july_2_blue', 'july_3_green', 'july_4_red', 'july_5_nearInfrared', 'july_6_shortInfrared1', 'july_7_shortInfrared2', 'july_evi2', 'july_nbr', 'july_ndmi', 'july_ndsi', 'july_ndvi', 'july_ndwi', 'august_1_ultraBlue', 'august_2_blue', 'august_3_green', 'august_4_red', 'august_5_nearInfrared', 'august_6_shortInfrared1', 'august_7_shortInfrared2', 'august_evi2', 'august_nbr', 'august_ndmi', 'august_ndsi', 'august_ndvi', 'august_ndwi', 'september_1_ultraBlue', 'september_2_blue', 'september_3_green', 'september_4_red', 'september_5_nearInfrared', 'september_6_shortInfrared1', 'september_7_shortInfrared2', 'september_evi2', 'september_nbr', 'september_ndmi', 'september_ndsi', 'september_ndvi', 'september_ndwi']
coordinates = ['POINT_X', 'POINT_Y']
absence = ['absence']
presence = ['presence']
response = ['response']
prediction = ['prediction']
output_columns = coordinates + absence + presence + response + prediction

In [ ]:
# Import packages for file manipulation, data manipulation, and plotting
import os
import numpy as np
import pandas as pd
# Import XGBoost gradient boosting implementations
from xgboost import XGBClassifier
from xgboost import XGBRegressor
# Import joblib
import joblib
# Import timing packages
import time
import datetime

In [ ]:
# Define a function to read threshold values from text file
def readThreshold(inFile):
    threshold_reader = open(inFile, "r")
    threshold = threshold_reader.readlines()
    threshold_reader.close()
    outThreshold = float(threshold[0])
    return outThreshold

In [ ]:
# Create a function to composite model results
def compositePrediction(input_data, presence, response, threshold):
    # Define a function to threshold absences and set presences equal to regression response
    def compositeRows(row):
        if row[presence[0]] < threshold:
            return 0
        elif row[presence[0]] >= threshold:
            return row[response[0]]
    # Apply function to all rows in test data
    input_data['prediction'] = input_data.apply(lambda row: compositeRows(row), axis=1)
    # Return the test data frame with composited results
    return input_data

In [ ]:
# Import the trained models
classifier = joblib.load(os.path.join(model_folder, 'classifier.joblib'))
regressor = joblib.load(os.path.join(model_folder, 'regressor.joblib'))

In [ ]:
# Read thresholds from text files in the workspace folder and store as variables
threshold = readThreshold(os.path.join(model_folder, 'threshold.txt'))

In [ ]:
# Create a list of input files for the prediction step
selected_watersheds = ['T1905040107', 'T1905040501', 'T1905040504', 'T1905040506', 'T1905040507', 'T1906010101', 'T1906010102', 'T1906010103', 'T1906010104', 'T1906010105', 'T1906010106', 'T1906010107', 'T1906010108', 'T1906010109', 'T1906010110', 'T1906010111', 'T1906010112', 'T1906010113', 'T1906010114', 'T1906010115', 'T1906010201', 'T1906010202', 'T1906010203', 'T1906010204', 'T1906010205', 'T1906010206', 'T1906010207', 'T1906010208', 'T1906010301', 'T1906010302', 'T1906010303', 'T1906010304', 'T1906010305', 'T1906010306', 'T1906010307', 'T1906010308', 'T1906010309', 'T1906010310', 'T1906010311', 'T1906010312', 'T1906010313', 'T1906010314', 'T1906010315', 'T1906010316', 'T1906020101', 'T1906020102', 'T1906020103', 'T1906020104', 'T1906020105', 'T1906020106', 'T1906020107', 'T1906020108', 'T1906020109', 'T1906020110', 'T1906020111', 'T1906020112', 'T1906020113', 'T1906020114', 'T1906020201', 'T1906020202', 'T1906020203', 'T1906020204', 'T1906020205', 'T1906020301', 'T1906020302', 'T1906020303', 'T1906020304', 'T1906020305', 'T1906020306', 'T1906020307', 'T1906020308', 'T1906020309', 'T1906020310', 'T1906020311', 'T1906020312', 'T1906020313', 'T1906020314', 'T1906020315', 'T1906020316', 'T1906020317', 'T1906020401', 'T1906020402', 'T1906020403', 'T1906020404', 'T1906020405', 'T1906020406', 'T1906020407', 'T1906020408', 'T1906020409', 'T1906020410', 'T1906020411', 'T1906020412', 'T1906020413', 'T1906020414', 'T1906020415', 'T1906020416', 'T1906020417', 'T1906020418', 'T1906020419', 'T1906020420', 'T1906020421', 'T1906020422', 'T1906020423', 'T1906020424', 'T1906020501', 'T1906020502', 'T1906020503', 'T1906020504', 'T1906020505', 'T1906020506', 'T1906020507', 'T1906020508', 'T1906020509', 'T1906020510', 'T1906020511', 'T1906020601', 'T1906020602', 'T1906020603', 'T1906020604', 'T1906020605', 'T1906020606', 'T1906020607', 'T1906020608', 'T1906020609', 'T1906030101', 'T1906030102', 'T1906030103', 'T1906030104', 'T1906030105', 'T1906030106', 'T1906030107', 'T1906030108', 'T1906030109', 'T1906030110', 'T1906030111', 'T1906030112', 'T1906030113', 'T1906030115', 'T1906030116', 'T1906030117', 'T1906030118', 'T1906030119', 'T1906030120', 'T1906030121', 'T1906030122', 'T1906030123', 'T1906030124', 'T1906030125', 'T1906030126', 'T1906030127', 'T1906030201', 'T1906030202', 'T1906030203', 'T1906030204', 'T1906030205', 'T1906030206', 'T1906030207', 'T1906030208', 'T1906030209', 'T1906030214', 'T1906030215', 'T1906030217', 'T1906030218', 'T1906030219', 'T1906030220', 'T1906030221', 'T1906030222', 'T1906030301', 'T1906030302', 'T1906030303', 'T1906030304', 'T1906030305', 'T1906030306', 'T1906030307', 'T1906030308', 'T1906030309', 'T1906030311', 'T1906030312', 'T1906030313', 'T1906030314', 'T1906030315', 'T1906030316', 'T1906030317', 'T1906030318', 'T1906030319', 'T1906030401', 'T1906030402', 'T1906030403', 'T1906030404', 'T1906030405', 'T1906030406', 'T1906030407', 'T1906030408', 'T1906030409', 'T1906030410', 'T1906030411', 'T1906030412', 'T1906030413', 'T1906030414', 'T1906040101', 'T1906040102', 'T1906040103', 'T1906040104', 'T1906040105', 'T1906040106', 'T1906040107', 'T1906040108', 'T1906040109', 'T1906040110', 'T1906040111', 'T1906040112', 'T1906040113', 'T1906040114', 'T1906040115', 'T1906040116', 'T1906040117', 'T1906040203', 'T1906040204', 'T1906040206', 'T1906040207', 'T1906040208', 'T1906040210', 'T1906040211', 'T1906040212', 'T1906040213', 'T1906040214', 'T1906040215', 'T1906040216', 'T1906040217', 'T1906040301', 'T1906040302', 'T1906040303', 'T1906040304', 'T1906040305', 'T1906040306', 'T1906040307', 'T1906040308', 'T1906040309', 'T1906040310', 'T1906040311', 'T1906050106', 'T1906050107', 'T1906050108', 'T1906050109', 'T1906050110', 'T1906050111', 'T1906050201', 'T1906050202', 'T1906050203', 'T1906050204', 'T1906050205', 'T1906050206', 'T1906050209', 'T1906050210', 'T1906050211', 'T1906050212', 'T1906050213', 'T1906050214', 'T1906050215', 'T1906050216', 'T1906050217', 'T1906050409', 'T1906050410', 'T1906050411', 'T1906050412', 'T1906050415', 'T1906050416', 'T1906050418', 'T1906050419']
selected_watersheds = [x + '.csv' for x in selected_watersheds]

In [ ]:
# Loop through the prediction function for all input files
count = 1
for watershed_data in selected_watersheds:
    print(f'Predicting watershed {count} of {len(selected_watersheds)}...')
    
    # Identify file path to the input csv file
    input_csv = os.path.join(watershed_folder, watershed_data)
    # Define the output csv file
    output_csv = os.path.join(prediction_folder, watershed_data)
    # Load the input data
    input_data = pd.read_csv(input_csv)
    input_data[predictor_all] = input_data[predictor_all].astype(int)
    
    # Define the X data
    X_data = input_data[predictor_all]
    
    # Predict the classifier
    print('\tClassifying presence-absence...')
    iteration_start = time.time()
    classification = classifier.predict_proba(X_data)
    # Concatenate predicted values to input data frame
    input_data['absence'] = classification[:,0]
    input_data['presence'] = classification[:,1]
    iteration_end = time.time()
    iteration_elapsed = int(iteration_end - iteration_start)
    iteration_success_time = datetime.datetime.now()
    print(f'\tCompleted at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
    print('\t----------')
    
    # Predict the regressor
    print('\tPredicting foliar cover...')
    iteration_start = time.time()
    regression = regressor.predict(X_data)
    # Concatenate predicted values to input data frame
    input_data['response'] = regression
    iteration_end = time.time()
    iteration_elapsed = int(iteration_end - iteration_start)
    iteration_success_time = datetime.datetime.now()
    print(f'\tCompleted at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
    print('\t----------')
    
    # Composite the classifier and regressor predictions
    print('\tExporting results...')
    iteration_start = time.time()
    input_data = compositePrediction(input_data, presence, response, threshold)
    # Export prediction to csv
    output_data = input_data[output_columns]
    output_data.to_csv(output_csv, header=True, index=False, sep=',', encoding='utf-8')
    iteration_end = time.time()
    iteration_elapsed = int(iteration_end - iteration_start)
    iteration_success_time = datetime.datetime.now()
    print(f'\tCompleted at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
    print('\t----------')
    
    count += 1